We will use "requests" to send request to our website. This process behaves as if we open the website on a browser but it saves the response of the website to an instance instead opening it graphically. We will be using the content of this response. "BeautifulSoup" helps us parse this content in a more useful way.

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def getSoup(website):
    sourceRequest = requests.get(website)
    soup = BeautifulSoup(sourceRequest.content , 'html.parser')
    return soup


You should check the website name and patterns in similar ones. In our example, lyrics page of an artist is formed in format of 'http://www.metrolyrics.com/artist-name-lyrics.html'. This type of patterns makes things easier for developers. In this case, you can change artist's name and see it works well for any artist.

In [3]:
%store -r artist
#artist='ayo teo'
artist=artist.lower()
print(artist)
artistName=artist.replace(' ', '-').replace('.', '')
artistPage = 'http://www.metrolyrics.com/' +artistName+ '-lyrics.html'
artistSoups=[]
artistSoup = getSoup(artistPage)
artistSoups.append(artistSoup)
isMultiplePaged=artistSoup.find('span', attrs={'class': 'pages'})
pageIndex=2
if isMultiplePaged is not None:
    while True:
        artistMultiplePage='http://www.metrolyrics.com/' +artistName+ '-alpage-' +str(pageIndex)+ '.html'
        artistMultipleSoup=getSoup(artistMultiplePage)
        if artistMultipleSoup==artistSoups[0]:
            break
        artistSoups.append(artistMultipleSoup)
        pageIndex+=1


ayo teo


I recommend you to print soup. If you are lucky, you'll see a well parsed source code of your website. In some cases, you might not see what you're looking in source code. Don't worry if that happens to you, I'll come to its solution later in the post. 

You need to inspect where your data is in the source code. I use Google Chrome for its "Inspect" feature to spot desired data in the source code. After this step, we are ready to use find and find_all functions in order to create lists of desired data.

I wanted to shrink our source code to tbody in "content" div just to not let the wrong data in. I'll use the song lyrics links so our code should look for 'a' elements in "content" section. Then, I'll be doing some string manipulation which I assume you are familiar with.

In [4]:
songLinkList={}
for artistSoup in artistSoups:
    contentTable=artistSoup.find('div', attrs={'class': 'content'})
    tableBody=contentTable.find('tbody')
    aList=tableBody.find_all('a')
    for a in aList:
        songLink=a['href']
        rawNameText=a['title']
        name=rawNameText.replace(artist.title(), '').replace('lyrics', '')[1:-1]
        songLinkList[name]=songLink


In [5]:
titleLyricsDict={}
for title, lyrics in songLinkList.items():
    lyricsSoup=getSoup(lyrics)
    lyrics=lyricsSoup.find_all('p', attrs={'class': 'verse'})
    lyricsList=''
    for verse in lyrics:
        desiredLyrics=verse.get_text().replace('\n', '[|]')+'[|]'
        lyricsList+=desiredLyrics
        lyricsList=lyricsList.replace('[|][|]', '[|]')
    titleLyricsDict[title]=lyricsList[:-3]
    

In [6]:
import csv
with open(artistName + '.csv', 'w') as f:
    for title, lyrics in titleLyricsDict.items():
        f.write("%s,%s\n"%(title, lyrics))
print(artistName + '.csv file created')

ayo-teo.csv file created
